In [ ]:
%matplotlib qt

In [ ]:
from main import *
import numpy as np

In [ ]:
q1 = Quadratic(
    np.array([[1, 0], [0, 1]]),
    np.array([1, 1]),
    -1.4
)

q2 = Quadratic(
    np.array([[0.1, 0], [0, 3]]),
    np.array([0, 0]),
    0
) # Dichotomy is much better

q3 = Quadratic(
    np.array([[4, 0], [0, 1]]),
    np.array([-1, 2]),
    1.5
)


def mf1(x, y):
    term1 = np.sin(x) * np.cos(y)
    term2 = -1.0 * np.exp(-(x**2 + y**2)/10)
    term3 = 0.1 * (x**2 + y**2)
    return term1 + term2 + term3


def mf2(x, y):
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2


def mf3(x, y):
    term2 = -1.0 * np.exp(-(x**2 + y**2)/10)
    return term2


def mf4(x, y):
    return (x**2 - 1)**2 + y**2 + 0.5 * x


def mf5(x, y):
    return (x**2 - 1)**2 + y**2


f1 = BiFuncCallableWrapper(mf1)
f2 = BiFuncCallableWrapper(mf2)
f3 = BiFuncCallableWrapper(mf3)
f4 = BiFuncCallableWrapper(mf4) # TOP 1
f5 = BiFuncCallableWrapper(mf5)

In [ ]:
func = BiFuncStatsDecorator(f4)
x_0 = np.array([1.0, 1.0])
PLOT_SIZE = 1.5

In [ ]:
import matplotlib.pyplot as plt


def plot_trajectory(trajectory: np.ndarray, title=None):
    # Create a meshgrid for the 3D plot
    x = np.linspace(-PLOT_SIZE, PLOT_SIZE, 100)
    y = np.linspace(-PLOT_SIZE, PLOT_SIZE, 100)
    X, Y = np.meshgrid(x, y)
    Z = np.zeros_like(X)

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i, j] = func(np.array([X[i, j], Y[i, j]]))

    # Plot the 3D surface
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.6)  # type: ignore

    # Plot the trajectory
    ax.plot(trajectory[:, 0], trajectory[:, 1], [func(np.array([x, y]))
            for x, y in trajectory], color='r', marker='o')

    if title:
        ax.set_title(title)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')  # type: ignore
    plt.show()

In [ ]:
def print_stats(func: BiFuncStatsDecorator, trajectory: np.ndarray, title=None):
    cc, gc = func.call_count, func.gradient_count
    print(f'Iterations: {len(trajectory) - 1}')
    print(f'x: {trajectory[-1]} f(x): {func(trajectory[-1])}')
    print(f'Function evaluations: {cc}')
    print(f'Gradient evaluations: {gc}')
    plot_trajectory(trajectory, title)
    func.reset()

In [ ]:
import math


def constant_h(c: float) -> LearningRateFunc:
    return lambda k: c


def geometric_h() -> LearningRateFunc:
    h0 = 1
    return lambda k: h0 / 2**k


def exponential_decay(λ: float) -> LearningRateFunc:
    assert λ > 0
    h0 = 1
    return lambda k: h0 * math.exp(-λ * k)


def polynomial_decay(α: float, β: float) -> LearningRateFunc:
    assert α > 0
    assert β > 0
    return lambda k: 1/math.sqrt(k + 1) * (β * k + 1) ** -α


def relative_x_condition(x: np.ndarray, prev: np.ndarray) -> bool:
    # ‖𝑥_{𝑘+1} − 𝑥_𝑘‖ < 𝜀(‖𝑥_{𝑘+1}‖ + 1)
    eps = 1e-9
    return bool(np.linalg.norm(x - prev) < eps * (np.linalg.norm(x) + 1))


def relative_f_condition(x: np.ndarray, prev: np.ndarray) -> bool:
    # ‖∇𝑓(𝑥_𝑘)‖^2 < 𝜀‖∇𝑓(𝑥_0)‖^2
    eps = 1e-9
    return bool(np.linalg.norm(func.gradient(x) ** 2) < eps * np.linalg.norm(func.gradient(x_0)) ** 2)


In [ ]:
h = exponential_decay(0.5)
h = polynomial_decay(0.5, 1)
h = geometric_h()
h = constant_h(0.01)

trajectory = learning_rate_scheduling(x_0, func, h, relative_x_condition)
print_stats(func, trajectory, "Learning rate scheduling")

In [ ]:
eps = 1e-9
trajectory = steepest_gradient_descent_dichotomy(
    x_0, func, eps, relative_x_condition)
print_stats(func, trajectory, "Dichotomy Gradient Descent")

In [ ]:
trajectory = steepest_gradient_descent_armijo(x_0, func, relative_x_condition)
print_stats(func, trajectory, "Armijo Gradient Descent")

In [ ]:
from scipy.optimize import fmin_cg

# Conjugate Gradient Descent, similar to steepest GD
fmin_cg(
    func,
    x_0 - 0.1,
    func.gradient,
    disp=True
)